In [1]:
# Importing the dependencies
import cv2
import os
import numpy as np

In [3]:
face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface.xml')
subjects = ['Charlie', 'Dalih', 'Nashmoni', 'Nychongsa', 'Pongjon', 'Chemkai']

In [5]:
# Function to detect faces in opencv
def detect_face(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #Converting the image into grayScale
    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5) #Detecting multiscale images; returns a list of faces
    if len(faces)==0:
        return None,None
    
    # Assuming that there is only a single face
    (x,y,w,h) = faces[0]
    
    return gray[y:y+w, x:x+h], faces[0] # Return only the face area

In [7]:
# Data preparation
def prepare_training_data(data_folder_path):
    # Getting the directories for each subject in the data folder
    dirs = os.listdir(data_folder_path)
    faces, labels = [], [] # Lists to hold the faces and labels of the dataset
    
    for dir_name in dirs: # Reading all the directories and going through the faces
        if not dir_name.startswith("S"): 
            continue #Ignoring the folders which aren't faces
        label = int(dir_name.replace("S",""))
        subject_dir_path = data_folder_path + "/" + dir_name 
        subject_images_names = os.listdir(subject_dir_path) # Getting the image names that are inside the directories
 
        #------STEP-3--------
        #go through each image name, read image, 
        #detect face and add face to list of faces
        for image_name in subject_images_names:
        #ignore system files like .DS_Store
            if image_name.startswith("."):
                continue
            #build image path
            #sample image path = training-data/s1/1.pgm
            image_path = subject_dir_path + "/" + image_name
    
            image = cv2.imread(image_path)
            #detect face
            face, rect = detect_face(image)
            if face is not None:
                #add face to list of faces
                faces.append(face)
                #add label for this face
                labels.append(label)
 
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
 
    return faces, labels

In [9]:
faces, labels = prepare_training_data("Dataset")
print("Data prepared")

#print total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

Data prepared
Total faces:  152
Total labels:  152


In [10]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

In [11]:
face_recognizer.train(faces, np.array(labels))

In [12]:
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
 
#function to draw text on give image starting from
#passed (x, y) coordinates. 
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)

In [13]:
def predict(test_img):
    #make a copy of the image as we don't want to change original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)

    #predict the image using our face recognizer 
    label = face_recognizer.predict(face)
    #get name of respective label returned by face recognizer
    label_text = subjects[label[0]-1]
    print(label_text)
 
    #draw a rectangle around face detected
    draw_rectangle(img, rect)
    #draw name of predicted person
    #draw_text(img, label_text, rect[0], rect[1]-5)
    cv2.putText(img, label_text, (rect[0], rect[1]-5),cv2.FONT_HERSHEY_PLAIN, 10, (0, 255, 0), 2)
    
    return img


In [ ]:
print("Predicting images...")

#load test images
test_img1 = cv2.imread("test_image.jpg")
test_img2 = cv2.imread("test_image1.jpg")

#perform a prediction
predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)
print("Prediction complete")

#display both images
im1 = cv2.resize(predicted_img1, (960, 540))
im2 = cv2.resize(predicted_img2, (960, 540))
cv2.imshow(subjects[1], im1)
cv2.imshow(subjects[2], im2)
cv2.waitKey(0)
cv2.destroyAllWindows()


Predicting images...
Chemkai
Nychongsa
Prediction complete
